In [53]:
import plotly.graph_objects as go
from pymongo import MongoClient, DESCENDING, ASCENDING
import pandas as pd
import numpy as np
import os, sys, time, datetime

## Add Time Interval

In [54]:
# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['historic_data']
collection = db['daily_stock_price']
# Get one symbol data from the collection
df = pd.DataFrame(list(collection.find({'symbol':'GIB'},{'_id': 0}).sort('date', ASCENDING)))
df

,date,symbol,low,open,close,high,volume
0,1998-10-07,GIB,6.125000,6.125000,6.218750,6.250000,39200
1,1998-10-08,GIB,5.968750,6.156250,6.000000,6.156250,19400
2,1998-10-09,GIB,5.250000,5.968750,5.562500,5.968750,88800
3,1998-10-12,GIB,5.625000,5.625000,5.625000,5.750000,8800
4,1998-10-13,GIB,5.406250,5.687500,5.406250,5.687500,18600
...,...,...,...,...,...,...,...
6535,2024-09-27,GIB,114.199997,116.669998,114.220001,116.669998,107000
6536,2024-09-30,GIB,113.010002,114.059998,114.959999,115.279999,83900
6537,2024-10-01,GIB,113.400002,115.000000,114.320000,115.279999,93400
6538,2024-10-02,GIB,113.730003,113.730003,115.129997,115.889999,133000


In [55]:
def create_new_interval_df(df, interval):
    new_df = df.set_index('date')
    new_df = new_df.groupby(pd.Grouper(freq=interval)).agg({'open': 'first',
                                                'high': 'max', 
                                                'low': 'min', 
                                                'close': 'last', 
                                                'volume': 'sum'})
    new_df.reset_index(inplace=True)
    new_df = new_df.dropna()
    return new_df

create_new_interval_df(df,'2D')

,date,open,high,low,close,volume
0,1998-10-07,6.125000,6.250000,5.968750,6.000000,58600
1,1998-10-09,5.968750,5.968750,5.250000,5.562500,88800
2,1998-10-11,5.625000,5.750000,5.625000,5.625000,8800
3,1998-10-13,5.687500,5.687500,5.250000,5.500000,36800
4,1998-10-15,5.500000,6.500000,5.500000,5.968750,80600
...,...,...,...,...,...,...
4741,2024-09-22,112.940002,113.559998,112.330002,113.510002,80000
4742,2024-09-24,113.779999,115.300003,113.150002,113.820000,188900
4743,2024-09-26,115.209999,116.669998,114.199997,114.220001,249900
4745,2024-09-30,114.059998,115.279999,113.010002,114.320000,177300


## Add Technical Features

In [ ]:
# Add bull and bear features in the dataframe
class add_features:
    def __init__(self, df):
        self.df = df.copy()

    def add_candlestick(self):
        self.df["BodyDiff"] = abs(self.df["open"] - self.df["close"])
        self.df["CandleStickType"] = np.where(self.df["open"] < self.df["close"], "green", "red")
        return self.df

    def continuous_increase(self, windows=3):
        for i in range(1, windows + 1):
            self.df[f"close_t-{i}"] = self.df["close"].shift(i)

        self.df['Incremental_High'] = (self.df['close'] > self.df['close_t-1']) \
                                        & (self.df['close_t-1'] > self.df['close_t-2']) \
                                        & (self.df['close_t-2'] > self.df['close_t-3'])
        return self.df

    def macd_golden_cross(self):
        self.df['MACD_GOLDEN_CROSS'] = (self.df['MACD'] > self.df['MACD_SIGNAL']) & (self.df['MACD'] < 0)
        return self.df

    def add_ema_band(self, threshold=0.05):
        self.df['169EMA_Upper'] = self.df['169EMA'] * (1 + threshold)
        self.df['169EMA_Lower'] = self.df['169EMA'] * (1 - threshold)
        return self.df
    
    # Add Technical Indicators
    def add_technical(self):
        
        # Add ema dual channels technical indicators
        self.df['8EMA'] = ta.trend.ema_indicator(self.df['close'], window=8)
        self.df['13EMA'] = ta.trend.ema_indicator(self.df['close'], window=13)
        self.df['144EMA'] = ta.trend.ema_indicator(self.df['close'], window=144)
        self.df['169EMA'] = ta.trend.ema_indicator(self.df['close'], window=169)

        # Add MACD technical indicator
        self.df['MACD'] = ta.trend.macd(self.df['close'], 
                                        window_slow=26, 
                                        window_fast=12)
        
        self.df['MACD_SIGNAL'] = ta.trend.macd_signal(self.df['close'], 
                                                    window_slow=26, 
                                                    window_fast=12, 
                                                    window_sign=9)
        
        self.df['MACD_HIST'] = ta.trend.macd_diff(self.df['close'], 
                                                window_slow=26, 
                                                window_fast=12, 
                                                window_sign=9)
        
        # Add ATR technical indicator
        self.df["atr"] = ta.volatility.AverageTrueRange(high=self.df.high, 
                                                        low=self.df.low, 
                                                        close=self.df.close).average_true_range()
        
        self.df["atr"] = self.df.atr.rolling(window=30).mean()
        
        return self.df
    
    def apply(self):
        self.add_technical()
        self.add_candlestick()
        self.continuous_increase()
        self.macd_golden_cross()
        self.add_ema_band()
        
        # Use last 1000 rows of the dataframe
        self.df = self.df.tail(3000) if len(self.df) > 3000 else self.df
        return self.df
    
testing_df = add_features(df).apply()

## Apply Strategy

In [ ]:
class Alert:
    
    def __init__(self, df):
        self.df = df.copy()
        self.df['MACD_Alert'] = -1  
        self.df['Engulf_Alert'] = -1 
        self.df['dual_channel_Alert'] = -1  
        self.df['382_Alert'] = -1  
        self.window = 3  # Number of days to compare the stock price
        
    def engulf_alert(self):
        
        # Previous Candle
        prev_open = self.df['open'].shift(1)
        prev_close = self.df['close'].shift(1)
        
        # Current Candle
        current_open = self.df['open']
        current_close = self.df['close']

        # Bullish Engulfing: current green candle engulfs previous red candle
        bullish_engulfing = (
            (prev_close < prev_open) &  # Previous candle was red
            (current_close > current_open) &  # Current candle is green
            (current_open < prev_close) & 
            (current_close > prev_open)
        )

        # Bearish Engulfing: current red candle engulfs previous green candle
        bearish_engulfing = (
            (prev_close > prev_open) &  # Previous candle was green
            (current_close < current_open) &  # Current candle is red
            (current_open > prev_close) & 
            (current_close < prev_open)
        )

        # Apply the conditions to the DataFrame
        self.df.loc[bullish_engulfing, 'Engulf_Alert'] = 1
        self.df.loc[bearish_engulfing, 'Engulf_Alert'] = 0
        
        return self.df

    def macd_alert(self):
        
        # Pre-compute conditions
        macd_above_signal = self.df['MACD'] > self.df['MACD_SIGNAL']
        macd_increasing = self.df['MACD'].diff() > 0
        macd_below_zero = self.df['MACD'] >= 0

        # Bullish condition
        bullish_macd = (
            macd_above_signal &
            macd_increasing&
            macd_below_zero
        )
        
        # Bearish condition
        bearish_macd = (
            (self.df['MACD'] < self.df['MACD_SIGNAL']) &
            (self.df['MACD'] > self.df['MACD'].shift(-1))  
        )

        # Apply the conditions to the DataFrame
        self.df.loc[bullish_macd, 'MACD_Alert'] = 1
        self.df.loc[bearish_macd, 'MACD_Alert'] = 0

        return self.df

    def dual_channel(self):
        
        # Confirm Technical Indicators Conditions
        ema_8_gt_ema_13 = self.df['8EMA'] > self.df['13EMA']
        ema_13_gt_ema_169 = self.df['13EMA'] > self.df['169EMA']
        ema_8_gt_ema_144 = self.df['8EMA'] > self.df['144EMA']
        
        # Confirm Close price action
        close_ge_ema_13 = self.df['close'] >= self.df['13EMA']
        close_ge_ema_144 = self.df['close'] >= self.df['144EMA']
        
        # Confirm Candle fully above 13 EMA
        low_ge_ema_13 = self.df['low'] >= self.df['13EMA'] 
        green_candle = self.df['CandleStickType'] == 'green'    
        
        # Confirm Candle in selected range
        open_in_slow_ema = self.df['open'].between(self.df['169EMA_Lower'], self.df['169EMA_Upper'])
        
        # Confirm Candle type
        green_candle = self.df['CandleStickType'] == 'green'
        
        # Bullish Conditions
        
        bullish_scenario_1 = (
            ema_8_gt_ema_13 & 
            ema_13_gt_ema_169 & 
            close_ge_ema_13 &
            low_ge_ema_13 &
            green_candle
        )
        
        # Bearish Condition
        bearish_scenario_1 = (
            (self.df['open'] < self.df['13EMA']) &
            (self.df['close'] < self.df['13EMA']) &
            (self.df['13EMA'] < self.df['8EMA'])
        )
        # Bullish Conditions 2
        bullish_scenario_2 = ( 
            ema_8_gt_ema_144 &
            ema_13_gt_ema_169 &
            close_ge_ema_144 &
            open_in_slow_ema &
            green_candle
            )
        
        # Bearish Condition 2
        bearish_scenario_2 = (
            (self.df['open'] < self.df['13EMA']) &
            (self.df['close'] < self.df['13EMA']) &
            (self.df['8EMA'] < self.df['13EMA']) &
            ~ open_in_slow_ema
        )

        # Apply conditions on different price scenarios
        
        self.df.loc[bullish_scenario_1 | bullish_scenario_2, 'dual_channel_Alert'] = 1
        self.df.loc[bearish_scenario_1 | bearish_scenario_2, 'dual_channel_Alert'] = 0
        
        return self.df

    def bullish_382_alert(self):
        # Calculate Fibonacci 38.2% retracement level
        diff = self.df['high'] - self.df['low']
        fib_382_level = self.df['high'] - 0.382 * diff
        open_above_fib_382 = self.df['open'] > fib_382_level
        
        # Apply condition
        self.df.loc[open_above_fib_382, '382_Alert'] = 1
        self.df.loc[~open_above_fib_382, '382_Alert'] = 0
        
        return self.df

    def add_alert(self):
        self.engulf_alert()
        self.macd_alert()
        self.dual_channel()
        self.bullish_382_alert()
        return self.df
    
# Apply the strategy
testing_df = Alert(testing_df).add_alert()

print(f"Bullish MACD Count: {len(testing_df[testing_df['MACD_Alert'] == 1])} | Bearish Count: {len(testing_df[testing_df['MACD_Alert'] == 0])}")
print(f"Bullish Engulf Count: {len(testing_df[testing_df['Engulf_Alert'] == 1])} | Bearish Engulf Count: {len(testing_df[testing_df['Engulf_Alert'] == 0])}")
print(f"Bullish 382 Count: {len(testing_df[testing_df['382_Alert'] == 1])} | Bearish 382 Count: {len(testing_df[testing_df['382_Alert'] == 0])}")
print(f"Bullish Dual Channel Count: {len(testing_df[testing_df['dual_channel_Alert'] == 1])} | Bearish Dual Channel Count: {len(testing_df[testing_df['dual_channel_Alert'] == 0])}")


Bullish MACD Count: 671 | Bearish Count: 1024
Bullish Engulf Count: 112 | Bearish Engulf Count: 118
Bullish 382 Count: 1218 | Bearish 382 Count: 1782
Bullish Dual Channel Count: 544 | Bearish Dual Channel Count: 860
